In [4]:
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [2]:
def load_images(data_dir):
    images = []
    labels = []
    for folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = preprocess_input(img_array)
            images.append(img_array)
            labels.append(folder)
    return np.array(images), np.array(labels)

In [3]:
data_dir = 'images'
images, labels = load_images(data_dir)

In [7]:
# print to check if labels are correctly created
# print(images,labels)

In [8]:
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_onehot = to_categorical(labels_encoded, num_classes=len(le.classes_))

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels_onehot, test_size=0.2, random_state=42)

In [10]:
# print to check if the sample is correctly created
# print(X_train, X_val, y_train, y_val)

In [11]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [12]:
# Add new layers for fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(le.classes_), activation='softmax')(x)


In [15]:
# Define the new model
model = Model(inputs=base_model.input, outputs=predictions)


In [16]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [17]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [19]:
# Define callbacks
checkpoint = ModelCheckpoint('fine_tuned_model.keras', monitor='val_loss', save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=10)

In [22]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=128, callbacks=[checkpoint, early_stop])

# Unfreeze some layers and fine-tune again
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compile the model again with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=128, callbacks=[checkpoint, early_stop])


Epoch 1/2
16/16 ━━━━━━━━━━━━━━━━━━━━ 177s 11s/step - accuracy: 1.0000 - loss: 6.4615e-05 - val_accuracy: 0.3413 - val_loss: 7.3107
Epoch 2/2
16/16 ━━━━━━━━━━━━━━━━━━━━ 175s 11s/step - accuracy: 1.0000 - loss: 5.1344e-05 - val_accuracy: 0.3453 - val_loss: 7.3672
Epoch 1/2
16/16 ━━━━━━━━━━━━━━━━━━━━ 174s 11s/step - accuracy: 1.0000 - loss: 2.4888e-05 - val_accuracy: 0.3353 - val_loss: 8.6718
Epoch 2/2
16/16 ━━━━━━━━━━━━━━━━━━━━ 177s 11s/step - accuracy: 1.0000 - loss: 8.2092e-06 - val_accuracy: 0.3373 - val_loss: 9.0180


In [24]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Assuming le is your LabelEncoder
le = LabelEncoder()
le.fit(labels)  # Replace `labels` with your actual labels variable

# Save the label encoder
joblib.dump(le, 'label_encoder.pkl')

['label_encoder.pkl']

In [25]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np 

# Load the fine-tuned model
model = load_model('fine_tuned_model.keras')

# Load the label encoder and labels
le = joblib.load('label_encoder.pkl')
labels_encoded = np.load('pokemon_labels.npy')

In [27]:
#  print to check if labels are encoded or not.
# labels_encoded

In [2]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np

# Load the fine-tuned model
model = load_model('fine_tuned_model.keras')

# Load the label encoder and labels
le = joblib.load('label_encoder.pkl')
labels_encoded = np.load('pokemon_labels.npy')


/Users/I561728/Documents/Personal Development/pokedex-model/pokenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
# prediction of pokemon 
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_pokemon(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_label = np.argmax(predictions, axis=1)[0]
    pokemon_name = le.inverse_transform([predicted_label])[0]
    return pokemon_name


In [15]:
# testing the model 
test_image_path = 'test/raichu.png'  # Replace with the path to a test image
predicted_pokemon = predict_pokemon(test_image_path)

print(f'Predicted Pokémon: {predicted_pokemon}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Predicted Pokémon: Tympole
